<a href="https://colab.research.google.com/github/ecealptekin/Skin-Cancer-Classification/blob/master/final_412_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Authentication**

In [0]:
!pip install PyDrive

In [0]:
#importing authentication libraries 

import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# **Download Data File**

In [0]:
#creating data file in drive

download = drive.CreateFile({'id': '1FKpO4xZE8bk1JgKi2DxTM_SPeOy90-3W'})


In [0]:
#downloading data file as zip

download.GetContentFile('Data_SkinCancer.zip')

In [0]:
#unzip downloaded data file

!unzip Data_SkinCancer.zip

Streaming output truncated to the last 5000 lines.
  inflating: Data_SkinCancer/Image_55.jpg  
  inflating: Data_SkinCancer/Image_550.jpg  
  inflating: Data_SkinCancer/Image_5500.jpg  
  inflating: Data_SkinCancer/Image_5501.jpg  
  inflating: Data_SkinCancer/Image_5502.jpg  
  inflating: Data_SkinCancer/Image_5503.jpg  
  inflating: Data_SkinCancer/Image_5504.jpg  
  inflating: Data_SkinCancer/Image_5505.jpg  
  inflating: Data_SkinCancer/Image_5506.jpg  
  inflating: Data_SkinCancer/Image_5507.jpg  
  inflating: Data_SkinCancer/Image_5508.jpg  
  inflating: Data_SkinCancer/Image_5509.jpg  
  inflating: Data_SkinCancer/Image_551.jpg  
  inflating: Data_SkinCancer/Image_5510.jpg  
  inflating: Data_SkinCancer/Image_5511.jpg  
  inflating: Data_SkinCancer/Image_5512.jpg  
  inflating: Data_SkinCancer/Image_5513.jpg  
  inflating: Data_SkinCancer/Image_5514.jpg  
  inflating: Data_SkinCancer/Image_5515.jpg  
  inflating: Data_SkinCancer/Image_5516.jpg  
  inflating: Data_SkinCancer/Imag

# **Download Image Files & Labels**


In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

Using TensorFlow backend.


In [0]:
#mount drive

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#loading train.csv from mounted drive

train = pd.read_csv('/content/drive/My Drive/Data/Train.csv')
train.head()

,Id,Category
0,Image_1,2
1,Image_2,2
2,Image_3,5
3,Image_4,2
4,Image_5,1


In [0]:
#decide fixed input shape to resize images
#images are in different sizes in original data file

#loading images from unzipped data file according to their image IDs
#resize images 
#grayscale set as False for RGB images 
#normalize images
#all training images in X array
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('Data_SkinCancer/'+str(train['Id'][i])+'.jpg', target_size=(28,28,1), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|██████████| 10000/10000 [01:43<00:00, 97.03it/s]


In [0]:
#loading label values

y=train['Category'].values
y = to_categorical(y)

In [0]:
#Visualize the dataset

import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
for i in range(20):
    plt.subplot(5,4, i+1)
    plt.xticks([])
    plt.imshow(X[i])
    plt.xlabel(y[i])
plt.show()

In [0]:
# number of classes

number_of_classes= train.Category.nunique()
print("Number of classes in the dataset: ", number_of_classes)

In [0]:
#showing the data instances per class
#proves data is imbalanced

import matplotlib.pyplot as plt; plt.rcdefaults()

objects = ('MEL', 'NV', 'BCC', 'AK', 'BKL')
y_pos = np.arange(number_of_classes)
unique, counts = np.unique(train['Category'].values, return_counts=True)
dict(zip(unique, counts))
performance = [counts[0],counts[1],counts[2],counts[3],counts[4]]

plt.bar(y_pos, performance, align='center', alpha=0.5, color="red")
plt.xticks(y_pos, objects)
plt.ylabel('Number of Samples')
plt.xlabel('Skin Cancer Types')
plt.title('Number of Samples per Class')

plt.show()

# **Data Augmentation**

In [0]:
#import skimage.io as io
#from skimage.transform import rotate, AffineTransform, warp
#from skimage.util import random_noise
#from skimage.filters import gaussian
#from tqdm import tqdm

#final_train_data = []
#final_target_train = []

#for i in tqdm(range(X.shape[0])):
#    final_train_data.append(X[i])
#    if y[i]!=2:
#      final_train_data.append(rotate(X[i], angle=45, mode = 'wrap'))
#      final_train_data.append(np.fliplr(X[i]))
#      final_train_data.append(np.flipud(X[i]))
#      final_train_data.append(random_noise(X[i],var=0.2**2))  
#      for j in range(5):
#        final_target_train.append(y[i])
#    else:
#      final_target_train.append(y[i])     

#len(final_train_data),len(final_target_train)

#y_final = to_categorical(final_target_train)
#final_train = np.array(final_train_data)

In [0]:
#visualize augmentation

#fig,ax = plt.subplots(nrows=1,ncols=5,figsize=(10,10))
#for i in range(5):
#    ax[i].imshow(final_train_data[i+29])
#    ax[i].axis('off')

In [0]:
#from collections import Counter

#print('Original dataset shape %s' % Counter(train['Category'].values))
#print('Augmented dataset shape %s' % Counter(final_target_train))

# **CNN Model**

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

In [0]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [0]:
model.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test))

Train on 8000 samples, validate on 2000 samples
Epoch 1/15
8000/8000 [==============================] - 24s 3ms/step - loss: 1.2624 - accuracy: 0.4784 - val_loss: 1.1894 - val_accuracy: 0.5175
Epoch 2/15
8000/8000 [==============================] - 24s 3ms/step - loss: 1.1699 - accuracy: 0.5256 - val_loss: 1.1633 - val_accuracy: 0.5380
Epoch 3/15
8000/8000 [==============================] - 24s 3ms/step - loss: 1.1135 - accuracy: 0.5544 - val_loss: 1.0788 - val_accuracy: 0.5820
Epoch 4/15
8000/8000 [==============================] - 27s 3ms/step - loss: 1.0752 - accuracy: 0.5721 - val_loss: 1.0553 - val_accuracy: 0.5885
Epoch 5/15
8000/8000 [==============================] - 24s 3ms/step - loss: 1.0524 - accuracy: 0.5838 - val_loss: 1.0354 - val_accuracy: 0.5915
Epoch 6/15
8000/8000 [==============================] - 24s 3ms/step - loss: 1.0305 - accuracy: 0.5969 - val_loss: 1.0199 - val_accuracy: 0.6030
Epoch 7/15
8000/8000 [==============================] - 24s 3ms/step - loss: 1.006

In [0]:
#making predictions

test = pd.read_csv('/content/drive/My Drive/Data/Test.csv')

In [0]:
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('Data_SkinCancer/'+str(test['Id'][i])+'.jpg', target_size=(28,28,1), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

100%|██████████| 5000/5000 [00:51<00:00, 96.24it/s]


In [0]:
prediction = model.predict_classes(test)

In [0]:
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test accuracy %.2f%%' % (test_acc*100))

2000/2000 [==============================] - 1s 701us/step
Test accuracy 64.00%


In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

In [0]:
result = pd.DataFrame(prediction, columns=['Category'] ).to_csv('/content/drive/My Drive/Data/30_may_22.16_pred.csv' , float_format = str, encoding = 'utf-8')